In [0]:
#!pip install eli5

In [0]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook #do wizualizacji postępów pracy

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/Matrix/DataWorkshopMatrix'

/content/drive/My Drive/Colab Notebooks/Matrix/DataWorkshopMatrix


In [0]:
df = pd.read_csv('data/men_shoes2.csv',low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
#df['brand_cat'] = df['brand'].factorize()[0]
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0] #brand z małej litery poprawia lekko wyniki

In [0]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0) #random_state aby wynik był powtarzalny
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [0]:
#df.features
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}

  if(str(x)=='nan') : return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()) )
len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan )

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_hat style', 'feat_eyewear frame style', 'feat_waist size',
       'feat_design', 'feat_waterproof', 'feat_gold karat',
       'feat_country/ organization', 'feat_sub style', 'feat_polarized',
       'feat_frame color'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys: 
  keys_stat[key] = df [False ==  df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100
#keys_stat
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer_cat'] = df['feat_manufacturer'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df [ df['feat_brand'] == df['brand'] ].shape

(8846, 531)

In [0]:
df [ df['feat_brand'] != df['brand'] ][['brand','feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [0]:
feats= ['']
model = RandomForestRegressor(max_depth=5, n_estimators=100)

In [0]:
run_model(['brand_cat'],model)

(-57.34162833183414, 4.167189200987787)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
#feats_cat

In [0]:
#df['weight'].unique() # do normalizzacji wszystko do gram

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_metal type_cat','feat_shape_cat']
#feats += feats_cat
#feats = lis(set(feats))
#feats += [ ]

result = run_model(feats,model)

In [0]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names = feats)

(-57.244903583729865, 4.227092222692045)


Weight,Feature
0.2595 ± 0.0110,brand_cat
0.1016 ± 0.0074,feat_material_cat
0.0233 ± 0.0022,feat_gender_cat
0.0173 ± 0.0009,feat_brand_cat
0.0125 ± 0.0008,feat_shape_cat
0.0097 ± 0.0012,feat_metal type_cat
0.0026 ± 0.0008,feat_style_cat


In [0]:
df['brand'].value_counts(normalize=True)

nike                      0.097210
puma                      0.033315
ralph lauren              0.028775
vans                      0.021116
new balance               0.020295
                            ...   
odor-eaters               0.000055
great northern            0.000055
ridge                     0.000055
under armour outerwear    0.000055
cabela's                  0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
df[df['brand']=='nike'].features_parsed.sample(10).values

array([{'sport': 'basketball', 'gender': 'men', 'shoe category': 'mens shoes', 'color': 'white/dark powder blue', 'casual & dress shoe style': 'basketball', 'manufacturer part number': '641444107', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'season': 'all-season', 'occasion': 'athletic', 'material': 'suede', 'gender': 'men', 'shoe size': '11.5', 'size': '11.5', 'color': 'black', 'model': '599277 006', 'manufacturer part number': '599277 006', 'brand': 'nike', 'age group': 'adult'},
       {'sport': 'skate shoes', 'occasion': 'skate shoes', 'material': 'suede', 'gender': 'men', 'shoe size': '8.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'suede', 'color': 'black', 'model': '704936 303', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'skateboarding shoes', 'manufacturer part number': '704936 303', 'variant group id': '224#mp#704936 303', 'brand': 'nike', 'age group': 'men', 's